## Abschlussproket
# Einführung in Python für Data Analytics
### Dozent: Dr. Alessandro Bramucci
### Bearbeitung und Präsentation: Jonathan Willrich, Gregor Henning, Sebastian Knoche

## Import Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

## Import data

```python
# speichere den relativen Pfad und den Dateinamen der Quelle 
file_besucher = 'data/besucher.csv'
file_geo = 'data/geo.txt'
file_kunden = 'data/kunden.csv'

# importiere Daten aus der Quelle in Dataframes
df_besucher = pd.read_csv(file_besucher, sep=";", decimal = ',')
df_geo = pd.read_table(file_geo, decimal = '.')
df_kunden = pd.read_csv(file_kunden, sep=";", decimal = '.')
```

In [2]:
# speichere den relativen Pfad und den Dateinamen der Quelle 
file_besucher = 'data/besucher.csv'
file_geo = 'data/geo.txt'
file_kunden = 'data/kunden.csv'

# importiere Daten aus der Quelle in Dataframes
df_besucher = pd.read_csv(file_besucher, sep=";", decimal = ',')
df_geo = pd.read_table(file_geo, decimal = '.')
df_kunden = pd.read_csv(file_kunden, sep=";", decimal = '.')

## Cleaning data

#### Analysis for df_besucher

In [3]:
df_besucher.isnull().sum()

Alter         0
Einkommen     0
Geschlecht    0
Zeit          0
KundeNr       0
dtype: int64

In [4]:
df_besucher.dtypes

Alter           int64
Einkommen     float64
Geschlecht      int64
Zeit          float64
KundeNr        object
dtype: object

In [5]:
df_besucher.head(5)

,Alter,Einkommen,Geschlecht,Zeit,KundeNr
0,30,39919.095335,0,29.0,K1814
1,41,53430.673086,0,27.0,K1544
2,64,73163.216927,0,32.0,K1945
3,41,36761.482581,0,44.0,K1825
4,35,42684.331632,1,33.0,K1144


#### Data modelation is not needed - df_besucher is in good shape

#### Analysis for df_geo

In [6]:
df_geo.isnull().sum()

KundeNr          0
Niederlassung    0
dtype: int64

In [7]:
df_geo.dtypes

KundeNr          object
Niederlassung    object
dtype: object

In [8]:
df_geo.head(5)

,KundeNr,Niederlassung
0,K0001,Sachsen
1,K0002,NRW
2,K0003,Hessen
3,K0004,Bayern
4,K0005,Bayern


#### Data modelation for df_geo

In [9]:
#for x in df_geo.index:
#    lower_niederlassung = df_geo.loc[x, 'Niederlassung'].lower()
#    if "berlin" in df_geo.loc[x, 'Niederlassung']:
#        print(df_geo.loc[x, 'Niederlassung'])
#        df_geo.loc[x, 'Niederlassung'] = "Berlin"
#        print(df_geo.loc[x, 'Niederlassung'])

```python
# Replace all instances of "NRW" in the column "Niederlassung" with "Nordrhein-Westfalen"
df_geo['Niederlassung'] = df_geo['Niederlassung'].apply(lambda x: 'Nordrhein-Westfalen' if 'NRW' in x else x)
# Replace all entries containing "Berlin" in the column "Niederlassung" with "Berlin"
df_geo['Niederlassung'] = df_geo['Niederlassung'].apply(lambda x: 'Berlin' if 'Berlin' in x else x)
```

In [10]:
# Replace all instances of "NRW" in the column "Niederlassung" with "Nordrhein-Westfalen"
df_geo['Niederlassung'] = df_geo['Niederlassung'].apply(lambda x: 'Nordrhein-Westfalen' if 'NRW' in x else x)
# Replace all entries containing "Berlin" in the column "Niederlassung" with "Berlin"
df_geo['Niederlassung'] = df_geo['Niederlassung'].apply(lambda x: 'Berlin' if 'Berlin' in x else x)

#### Analysis for df_kunden

In [11]:
df_kunden.isnull().sum()

Alter         0
Einkommen     0
Preis         0
Geschlecht    5
Zeit          0
KundeNr       0
dtype: int64

In [12]:
df_kunden.dtypes

Alter           int64
Einkommen     float64
Preis         float64
Geschlecht    float64
Zeit          float64
KundeNr        object
dtype: object

In [13]:
df_kunden.head(5)

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr
0,64,66894.0,88160.31,1.0,43.0,K0310
1,54,77644.0,103145.70,1.0,40.0,K1042
2,55,44341.0,80565.16,0.0,37.0,K0382
3,49,67271.0,83949.89,0.0,42.0,K0498
4,46,49832.0,93781.58,0.0,41.0,K0552


#### Data modelation for df_kunden

```python
# Find the mode for the values in the column "Geschlecht"
geschlecht_mode = df_kunden['Geschlecht'].mode()[0]
```

In [14]:
# Calculate the mode for the values in the column "Geschlecht"
geschlecht_mode = df_kunden['Geschlecht'].mode()[0]
print("The mode is "+str(geschlecht_mode))

The mode is 1.0


```python
# Replace all empty entries in the column "Geschlecht" with the mode of the values in the column "Geschlecht" 
df_kunden['Geschlecht'].fillna(geschlecht_mode, inplace = True)
# Convert column "Geschlecht" to the more propper integer date type
df_kunden['Geschlecht'] = df_kunden['Geschlecht'].astype(int)
```

In [15]:
# Replace all empty entries in the column "Geschlecht" with the mode of the values in the column "Geschlecht" 
df_kunden['Geschlecht'].fillna(geschlecht_mode, inplace = True)
# Convert column "Geschlecht" to the more propper integer date type
df_kunden['Geschlecht'] = df_kunden['Geschlecht'].astype(int)

#### Further data modeling for df_kunden, to remove extreme values

```python
# Berechnung von Q1 (25. Perzentil), Q3 (75. Perzentil) und IQR
Q1 = df_kunden['Einkommen'].quantile(0.25)
Q3 = df_kunden['Einkommen'].quantile(0.75)
IQR = Q3 - Q1

# Berechnung der Ausreißer-Grenzen
untere_grenze = Q1 - 1.5 * IQR
obere_grenze = Q3 + 1.5 * IQR

# Identifizierung von Ausreißern
df_kunden['Ausreißer'] = (df_kunden['Einkommen'] < untere_grenze) | (df_kunden['Einkommen'])
```

In [16]:
# Berechnung von Q1 (25. Perzentil), Q3 (75. Perzentil) und IQR
Q1 = df_kunden['Einkommen'].quantile(0.25)
Q3 = df_kunden['Einkommen'].quantile(0.75)
IQR = Q3 - Q1

# Berechnung der Ausreißer-Grenzen
untere_grenze = Q1 - 1.5 * IQR
obere_grenze = Q3 + 1.5 * IQR

# Identifizierung von Ausreißern
df_kunden['Ausreißer'] = (df_kunden['Einkommen'] < untere_grenze) | (df_kunden['Einkommen'] > obere_grenze)

```python
# Calculate the median for the values in the column "Einkommen"
eink_medianwert = df_kunden['Einkommen'].median()
```

In [17]:
# Calculate the median for the values in the column "Einkommen"
eink_medianwert = df_kunden['Einkommen'].median()
print("The median is "+str(eink_medianwert))

The median is 59013.0


```python
# Replace extreme values with the median of the column "Einkommen"
df_kunden.loc[df_kunden['Ausreißer'] == True, 'Einkommen'] = eink_medianwert
```

In [18]:
# Replace extreme values with the median of the column "Einkommen"
df_kunden.loc[df_kunden['Ausreißer'] == True, 'Einkommen'] = eink_medianwert

# Vielen Dank für Ihre Aufmerksamkeit 🐍 